In [3]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns

import acquire
import prepare

# This is to make sure matplotlib doesn't throw the following error:
# The next line fixes "TypeError: float() argument must be a string or a number, not 'Timestamp' matplotlib"
pd.plotting.register_matplotlib_converters()

In [2]:
df = pd.read_csv('anonymized-curriculum-access-07-2021.txt',
                      engine='python',
                     header=None,
                     index_col=False,
                     sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                     na_values='"-"',
                     )

In [3]:
df.columns = ['date','time','page_viewed','user_id','cohort_id','ip']

In [4]:
df['cohort_id'] = df.cohort_id.fillna(0)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018810 entries, 0 to 1018809
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   date         1018810 non-null  object 
 1   time         1018810 non-null  object 
 2   page_viewed  1018809 non-null  object 
 3   user_id      1018810 non-null  object 
 4   cohort_id    1018810 non-null  float64
 5   ip           1018809 non-null  object 
dtypes: float64(1), object(5)
memory usage: 46.6+ MB


In [6]:
# Drop the nulls
df = df.dropna()

In [7]:
# Create a datetime column by concatenating date and time
#df.index = pd.to_datetime(df.date + " " + df.time)

In [8]:
df.head()

,date,time,page_viewed,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [9]:
#df = df.drop(['date', 'time'], axis = 1)

In [10]:
df.head()

,date,time,page_viewed,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [11]:
id_counts = df.groupby(['user_id'])['date', 'cohort_id','page_viewed'].nunique()
cohort_counts = df.groupby(['cohort_id'])['page_viewed','user_id', 'date'].nunique()
first_access = df.groupby(['user_id'])['date'].min()
id_by_first_access_date = pd.DataFrame({'first_access_date': first_access}).reset_index().groupby('first_access_date').count()

In [12]:
id_by_first_access_date

,user_id
first_access_date,
2018-01-26,50
2018-01-27,2
2018-01-28,2
2018-01-29,12
2018-01-30,7
...,...
2021-06-28,23
2021-07-04,2
2021-07-06,2


### Which lessons are least accessed?

In [115]:
page_views = df.groupby(['page_viewed'])['user_id'].agg(['count','nunique'])
observed = page_views.sort_values(by = 'count', ascending = True)
observed.head(15)

,count,nunique
page_viewed,,
%20https://github.com/RaulCPena,1,1
cohorts/24/capstone-all,1,1
cohorts/24/grades,1,1
cohorts/27/quizzes,1,1
content/appendix/control-structures.html,1,1
content/appendix/javascript/functions/controllers.html,1,1
content/appendix/javascript/functions/models.html,1,1
content/appendix/javascript/functions/scope.html,1,1
content/appendix/javascript/functions/templating.html,1,1


In [116]:
observed = observed.reset_index()

In [117]:
observed = observed[~(observed['page_viewed'].str.contains('appendix|cohorts|examples|caps|github|coding-challenges \
                                                            |advanced-topics|extra|jpeg|ico|csv|project'))]

In [118]:
observed = observed[observed['page_viewed'].str.contains('/')]

In [119]:
observed.shape

(1197, 3)

Let's get rid of the noise by calculating the upper and lower bounds to locate outliers

In [120]:
def outlier_calculation(df, variable):
    '''
    calcualtes the lower and upper bound to locate outliers in variables
    '''
    quartile1, quartile3 = np.percentile(df[variable], [25,75])
    IQR_value = quartile3 - quartile1
    lower = quartile1 - (1.5 * IQR_value)
    upper = quartile3 + (1.5 * IQR_value)
    '''
    returns the lowerbound and upperbound values
    '''
    print(f'For {variable} the lower bound is {lower} and  upper bound is {upper}')
    df = df[(df[variable] > lower) & (df[variable] < upper)]
    return df

In [121]:
outlier_calculation(observed, 'count')

For count the lower bound is -418.5 and  upper bound is 705.5


,page_viewed,count,nunique
10,content/conditionals.html,1,1
11,content/control-structures-ii,1,1
26,coding-challenges/professional,1,1
47,classification/knn.md,1,1
48,cli/4-navigating-the-filesystem,1,1
...,...,...,...
2194,regression/feature-engineering,667,76
2195,python/functions,667,103
2196,sql/basic-statements,670,97
2197,python/intro-to-matplotlib,685,87


In [129]:
observed['count'].eq(1).sum()

200

#### Top 5 of least accessed lessons

In [131]:
# lower bound absolute 418 - 200 (number of count = 1) = 218
observed.loc[observed['count'] > 218].head(5)

,page_viewed,count,nunique
1944,2-storytelling/2.2-create,219,50
1945,4-python/8.2-intro-to-matplotlib,219,32
1948,10-anomaly-detection/3-discrete-probabilistic-...,223,42
1949,4-python/7.4.2-series,224,26
1951,8-clustering/5.1-kmeans-part-1,227,34


### Are there students who, when active, hardly accessed the curriculum? If so, what information do you have about these students?

In [5]:
sql_query ='''
SELECT * FROM cohorts;
'''


# *************************************  connection url **********************************************

# Create helper function to get the necessary connection url.
def get_connection(db_name):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    from env import host, username, password
    return f'mysql+pymysql://{username}:{password}@{host}/{db_name}'

# ************************************ generic acquire function ***************************************************************

def get_data_from_sql(db_name, query ):
    """
    This function takes in a string for the name of the database that I want to connect to
    and a query to obtain my data from the Codeup server and return a DataFrame.
    db_name : df name in a string type
    query: aalready created query that was named as query 
    Example:
    query = '''
    SELECT * 
    FROM table_name;
    '''
    df = get_data_from_sql('zillow', query)
    """
    df = pd.read_sql(query, get_connection(db_name))
    return df


# ************************************ unique values ***************************************************************

def report_unique_val (df):
    '''
    takes in a df and gives you a report of number of unique values and count values <15 (categorical)
    count values <15 (numerical)
    '''
    num_cols = df.select_dtypes(exclude = 'O').columns.to_list()
    cat_cols = df.select_dtypes(include = 'O').columns.to_list()
    for col in df.columns:
            print(f'**{col}**')
            le = df[col].nunique()
            print ('Unique Values : ', df[col].nunique())
            print(' ')
            if col in cat_cols and le < 15:
                print(df[col].value_counts())
            if col in num_cols and  le < 23:
                 print(df[col].value_counts().sort_index(ascending=True)) 
            elif col in num_cols and le <150:
                print(df[col].value_counts(bins=10, sort=False).sort_index(ascending=True))
            elif col in num_cols and le <1001:
                print(df[col].value_counts(bins=100, sort=False).sort_index(ascending=True))

            print('=====================================================')






# ************************************ curriculum_logs ***************************************************************


#acquire function
def acquire ():
    #aquire first data frame
    colnames = ['date', 'endpoint', 'user_id', 'cohort_id', 'source_ip']
    df1 = pd.read_csv("anonymized-curriculum-access-07-2021.txt", 
                 sep="\s", 
                 header=None, 
                 names = colnames, 
                 usecols=[0, 2, 3, 4, 5])
    
    #acquire second df
    df2 = get_data_from_sql('curriculum_logs', sql_query)
    #drop columns that we don't need
    df2 = df2.drop(columns =[ 'deleted_at', 'slack'])
    
    #merge
    df = df1.merge(df2, left_on='cohort_id', right_on= 'id', how = 'left')
    
    # drop id because it is duplicated
    df.drop(columns= ['id'],inplace = True)
    
    return df

In [6]:
df = acquire()

In [8]:
df.head()

,date,endpoint,user_id,cohort_id,source_ip,name,start_date,end_date,created_at,updated_at,program_id
0,2018-01-26,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
1,2018-01-26,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
2,2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
3,2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
4,2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0


In [18]:
df.date = pd.to_datetime(df.date)

In [20]:
df.start_date = pd.to_datetime(df.start_date)

In [21]:
df.end_date = pd.to_datetime(df.end_date)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1018810 entries, 0 to 1018809
Data columns (total 11 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   date        1018810 non-null  datetime64[ns]
 1   endpoint    1018809 non-null  object        
 2   user_id     1018810 non-null  int64         
 3   cohort_id   965313 non-null   float64       
 4   source_ip   1018810 non-null  object        
 5   name        954313 non-null   object        
 6   start_date  954313 non-null   datetime64[ns]
 7   end_date    954313 non-null   datetime64[ns]
 8   created_at  954313 non-null   object        
 9   updated_at  954313 non-null   object        
 10  program_id  954313 non-null   float64       
dtypes: datetime64[ns](3), float64(2), int64(1), object(5)
memory usage: 93.3+ MB


In [23]:
# Create a column to determine if the student was active or not when they accessed the curriculum
df['is_active'] = (df.date > df.start_date) & (df.date < df.end_date)

In [32]:
df.head()


,date,endpoint,user_id,cohort_id,source_ip,name,start_date,end_date,created_at,updated_at,program_id,is_active
0,2018-01-26,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,False
1,2018-01-26,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,False
2,2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,False
3,2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,False
4,2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,True


In [54]:
# Separate out the students who were active when they accessed the curriculum
df2 = df[(df['is_active'])]

In [55]:
df2 

,date,endpoint,user_id,cohort_id,source_ip,name,start_date,end_date,created_at,updated_at,program_id,is_active
4,2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,True
5,2018-01-26,javascript-i/loops,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,True
6,2018-01-26,javascript-i/conditionals,3,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,True
7,2018-01-26,javascript-i/functions,3,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,True
8,2018-01-26,javascript-i/loops,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1018805,2021-07-15,toc,925,138.0,24.26.246.133,Neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,2.0,True
1018806,2021-07-15,java-iii/finish-the-adlister,925,138.0,24.26.246.133,Neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,2.0,True
1018807,2021-07-15,java-ii/arrays,933,138.0,72.190.28.51,Neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,2.0,True
1018808,2021-07-15,java-ii/object-oriented-programming,933,138.0,72.190.28.51,Neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,2.0,True


In [56]:
df2 = df2.sort_values('user_id')

In [57]:
df2

,date,endpoint,user_id,cohort_id,source_ip,name,start_date,end_date,created_at,updated_at,program_id,is_active
4,2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,True
18191,2018-03-04,java-ii,2,22.0,67.11.104.47,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,True
18165,2018-03-04,java-ii/arrays,2,22.0,67.11.104.47,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,True
18164,2018-03-04,java-ii/object-oriented-programming,2,22.0,67.11.104.47,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,True
27272,2018-03-18,/,2,22.0,67.11.104.47,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1015162,2021-07-13,/,1025,135.0,136.50.98.51,Marco,2021-01-25,2021-07-19,2021-01-20 21:31:11,2021-01-20 21:31:11,2.0,True
1006057,2021-07-06,/,1025,135.0,136.50.98.51,Marco,2021-01-25,2021-07-19,2021-01-20 21:31:11,2021-01-20 21:31:11,2.0,True
1015163,2021-07-13,mysql,1025,135.0,136.50.98.51,Marco,2021-01-25,2021-07-19,2021-01-20 21:31:11,2021-01-20 21:31:11,2.0,True
967328,2021-06-09,/,1025,135.0,136.50.98.51,Marco,2021-01-25,2021-07-19,2021-01-20 21:31:11,2021-01-20 21:31:11,2.0,True


In [60]:
# What are the student_ids who hardly accessed the curriculum while being an active student?
df2['user_id'].value_counts().tail(5)

956    5
278    4
832    3
679    3
879    1
Name: user_id, dtype: int64

#### These students hardly accessed the curriculum.

In [61]:
df2.loc[df2['user_id'] == 956]

,date,endpoint,user_id,cohort_id,source_ip,name,start_date,end_date,created_at,updated_at,program_id,is_active
891694,2021-04-15,toc,956,139.0,162.200.114.251,Oberon,2021-04-12,2021-10-01,2021-04-12 18:07:21,2021-04-12 18:07:21,2.0,True
891690,2021-04-15,/,956,139.0,162.200.114.251,Oberon,2021-04-12,2021-10-01,2021-04-12 18:07:21,2021-04-12 18:07:21,2.0,True
891724,2021-04-15,javascript-i/introduction/primitive-types,956,139.0,162.200.114.251,Oberon,2021-04-12,2021-10-01,2021-04-12 18:07:21,2021-04-12 18:07:21,2.0,True
891897,2021-04-15,javascript-i/introduction/operators,956,139.0,162.200.114.251,Oberon,2021-04-12,2021-10-01,2021-04-12 18:07:21,2021-04-12 18:07:21,2.0,True
891710,2021-04-15,javascript-i/introduction/operators,956,139.0,162.200.114.251,Oberon,2021-04-12,2021-10-01,2021-04-12 18:07:21,2021-04-12 18:07:21,2.0,True


In [62]:
df2.loc[df2['user_id'] == 278]

,date,endpoint,user_id,cohort_id,source_ip,name,start_date,end_date,created_at,updated_at,program_id,is_active
131804,2018-09-27,java-ii/collections,278,24.0,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,2018-05-25 22:25:57,2018-05-25 22:25:57,2.0,True
131788,2018-09-27,java-ii/arrays,278,24.0,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,2018-05-25 22:25:57,2018-05-25 22:25:57,2.0,True
131802,2018-09-27,java-ii/arrays,278,24.0,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,2018-05-25 22:25:57,2018-05-25 22:25:57,2.0,True
131699,2018-09-27,/,278,24.0,97.105.19.58,Voyageurs,2018-05-29,2018-10-11,2018-05-25 22:25:57,2018-05-25 22:25:57,2.0,True


In [63]:
df2.loc[df2['user_id'] == 832]

,date,endpoint,user_id,cohort_id,source_ip,name,start_date,end_date,created_at,updated_at,program_id,is_active
754195,2020-12-07,/,832,62.0,69.154.52.98,Jupiter,2020-09-21,2021-03-30,2020-09-21 18:06:27,2020-09-21 18:06:27,2.0,True
754204,2020-12-07,javascript-i,832,62.0,69.154.52.98,Jupiter,2020-09-21,2021-03-30,2020-09-21 18:06:27,2020-09-21 18:06:27,2.0,True
754206,2020-12-07,html-css,832,62.0,69.154.52.98,Jupiter,2020-09-21,2021-03-30,2020-09-21 18:06:27,2020-09-21 18:06:27,2.0,True


In [64]:
df2.loc[df2['user_id'] == 679]

,date,endpoint,user_id,cohort_id,source_ip,name,start_date,end_date,created_at,updated_at,program_id,is_active
597687,2020-07-14,1-fundamentals/modern-data-scientist.jpg,679,59.0,24.28.146.155,Darden,2020-07-13,2021-01-12,2020-07-13 18:32:19,2020-07-13 18:32:19,3.0,True
597685,2020-07-14,1-fundamentals/1.1-intro-to-data-science,679,59.0,24.28.146.155,Darden,2020-07-13,2021-01-12,2020-07-13 18:32:19,2020-07-13 18:32:19,3.0,True
597686,2020-07-14,1-fundamentals/AI-ML-DL-timeline.jpg,679,59.0,24.28.146.155,Darden,2020-07-13,2021-01-12,2020-07-13 18:32:19,2020-07-13 18:32:19,3.0,True


In [65]:
df2.loc[df2['user_id'] == 879]

,date,endpoint,user_id,cohort_id,source_ip,name,start_date,end_date,created_at,updated_at,program_id,is_active
799236,2021-01-26,/,879,135.0,136.50.50.187,Marco,2021-01-25,2021-07-19,2021-01-20 21:31:11,2021-01-20 21:31:11,2.0,True


### Takeaway - It is most likely these students did not complete the course.